# Landmark Classifier

Photo sharing and photo storage services like to have location data for each photo that is uploaded. With the location data, these services can build advanced features, such as automatic suggestion of relevant tags or automatic photo organization, which help provide a compelling user experience. Although a photo’s location can often be obtained by looking at the photo’s metadata, many photos uploaded to these services will not have location metadata available. This can happen when, for example, the camera capturing the picture does not have GPS or if a photo’s metadata is scrubbed due to privacy concerns.

If no location metadata for an image is available, one way to infer the location is to detect and classify a discernible landmark in the image. Given the large number of landmarks across the world and the immense volume of images that are uploaded to photo sharing services, using human judgement to classify these landmarks would not be feasible.

This notebook classifies 50 landmarks by building models to automatically predict the location of the image based on any landmarks depicted in the image.

### Table of Content

In [1]:
# import all the packages
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from torchsummary import summary

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

from utils import models, preprocess

In [2]:
# define the batch size
BATCH_SIZE = 64

# get the loaders
train_loader, val_loader, test_loader, dataset = preprocess.create_loaders(n_batch=BATCH_SIZE)

In [3]:
# store loaders
loaders = {'train': train_loader,
           'val':   val_loader,
           'test':  test_loader}

# get dictionary and descriptions
dictionary = preprocess.Dictionary(dataset)

# show first three instances
dictionary.simple_print(3)

						Classes & Indexes
0:	Haleakala National Park
1:	Mount Rainier National Park
2:	Ljubljana Castle


In [4]:
preprocess.visualize(dictionary, train_loader, single=True)

TypeError: forward() got an unexpected keyword argument 'image'